In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from crud import AnimalShelter
from helpers import build_query, filter_and_sort_data, build_breed_index, build_age_bst, BST
from dash.exceptions import PreventUpdate
from dash import ctx


###########################
# Data Manipulation / Model
###########################

# Database credentials 
username = "aacuser"
password = "guest123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# Remove the MongoDB _id column to prevent compatibility issues in Dash tables
df.drop(columns=['_id'],inplace=True)

from helpers import build_breed_index
breed_index = build_breed_index(df)

# Build BST from original dataset
age_bst = build_age_bst(df)

# Valid users
VALID_USERS = {
    'admin': 'admin123',
    'aacuser': 'guest123'
}

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)
app.config.suppress_callback_exceptions = True

# Encode the logo image in Base64 format for embedding in Dash
with open('Grazioso Salvare Logo.png', 'rb') as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode()

# Layout of Login Screen
login_layout = html.Div([
    html.Div([
        html.H1("Welcome to Grazioso Salvare", style={'textAlign': 'center', 'marginBottom': '30px'}),
        
        html.Label("Username", style={'fontWeight': 'bold'}),
        dcc.Input(
            id='login-username',
            type='text',
            placeholder='Enter username',
            style={'width': '90%', 'padding': '10px', 'marginBottom': '15px'}
        ),

        html.Label("Password", style={'fontWeight': 'bold'}),
        dcc.Input(
            id='login-password',
            type='password',
            placeholder='Enter password',
            n_submit=0,
            style={'width': '90%', 'padding': '10px', 'marginBottom': '15px'}
        ),

        html.Button('Login', id='login-button', n_clicks=0,
                    style={
                        'width': '100%',
                        'padding': '10px',
                        'backgroundColor': '#007BFF',
                        'color': 'white',
                        'border': 'none',
                        'borderRadius': '5px',
                        'cursor': 'pointer',
                        'fontWeight': 'bold'
                    }),

        html.Div(id='login-output', style={'color': 'red', 'marginTop': '15px', 'textAlign': 'center'})
    ], style={
        'width': '300px',
        'margin': '100px auto',
        'padding': '30px',
        'border': '1px solid #ccc',
        'borderRadius': '10px',
        'boxShadow': '0 0 10px rgba(0, 0, 0, 0.1)',
        'boxSizing': 'border-box'
    })
])

app.layout = html.Div(id='main-layout', children=[login_layout])

def create_dashboard_layout():
    return html.Div([
        #Header
        html.Div([
            html.A(
                html.Img(
                    src=f"data:image/png;base64,{encoded_image}",
                    style={'height': '200px', 'width': 'auto'}
                ),
                href="https://www.snhu.edu",
                target="_blank"
            ),
            html.H1("Grazioso Salvare Dashboard", style={'textAlign': 'center'}),
            html.H2("Enhanced by Matthew Guarino - 8/2/2025", style={'textAlign': 'center'}),
            html.H3("Created by Matthew Guarino - 2/20/2025", style={'textAlign': 'center'})
        ], style={'textAlign': 'center'}),
        
#Filters Row    
html.Div([
    html.Div([
        html.Label("Rescue Type:", style={'font-weight': 'bold', 'margin-right': '10px'}),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All', 'value': 'All'},
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster/Tracking', 'value': 'Disaster'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='All',
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ], style={'margin-right': '40px'}),

    html.Div([
        html.Label("Filter by Breed:", style={'font-weight': 'bold', 'margin-right': '6px'}),
        dcc.Dropdown(
            id='breed-dropdown',
            placeholder='Select a breed...',
            style={'width': '300px'}
        )
    ], style={'margin-right': '40px'}),

    html.Div([
        html.Label("Sort by:", style={'font-weight': 'bold', 'margin-right': '6px'}),
        dcc.Dropdown(
            id='sort-dropdown',
            options=[
                {'label': 'Name', 'value': 'name'},
                {'label': 'Breed', 'value': 'breed'},
                {'label': 'Age (weeks)', 'value': 'age_upon_outcome_in_weeks'}
            ],
            placeholder='Select a field to sort by',
            style={'width': '200px'}
        )
    ])
], style={
    'display': 'flex',
    'flex-direction': 'row',
    'align-items': 'center',
    'gap': '20px',
    'flex-wrap': 'nowrap',
    'margin': '20px 0'}),
        
# Age Slider    
html.Div([
    html.Label("Filter by Age (weeks):"),
    dcc.RangeSlider(
        id='age-range-slider',
        min=int(df['age_upon_outcome_in_weeks'].min()),
        max=int(df['age_upon_outcome_in_weeks'].max()),
        step=1,
        marks=None,
        tooltip={"placement": "bottom", "always_visible": True},
        value=[
            int(df['age_upon_outcome_in_weeks'].min()),
            int(df['age_upon_outcome_in_weeks'].max())
        ]
    )
], style={'margin': '20px'}),
        
#Export Buttons
html.Div([
    html.Button("Download CSV", id="btn_csv", n_clicks=0),
    html.Button("Download JSON", id="btn_json", n_clicks=0),
    dcc.Download(id="download-dataframe-csv"),
    dcc.Download(id="download-dataframe-json")
], style={'margin': '20px'}),    

# Data Table for displaying filtered dog data
dash_table.DataTable(id='datatable-id', 
    columns=[
        {'name': 'name', 'id': 'name', 'type': 'text'},
        {'name': 'breed', 'id': 'breed', 'type': 'text'},
        {'name': 'age_upon_outcome_in_weeks', 'id': 'age_upon_outcome_in_weeks', 'type': 'numeric'},
        {'name': 'sex_upon_outcome', 'id': 'sex_upon_outcome', 'type': 'text'},
        {'name': 'location_lat', 'id': 'location_lat', 'type': 'numeric'},
        {'name': 'location_long', 'id': 'location_long', 'type': 'numeric'}
    ], 
    page_size=10, 
    style_table={'overflowX': 'auto'}, 
    row_selectable='single'),

html.Div([
        html.Div(id='graph-id', style={'width': '50%', 'display': 'inline-block'}),
        html.Div(id='map-id', style={'width': '50%', 'display': 'inline-block'})
    ], style={'display': 'flex'})
    ])

@app.callback(
    [Output('main-layout', 'children'),
     Output('login-output', 'children')],
    [Input('login-button', 'n_clicks'),
     Input('login-password', 'n_submit')],
    [State('login-username', 'value'),
     State('login-password', 'value')],
    prevent_initial_call=True
)
def handle_login(n_clicks, n_submit, username, password):
    if not username or not password:
        return login_layout, "Please enter username and password"
    
    if username in VALID_USERS and VALID_USERS[username] == password:
        return create_dashboard_layout(), ""
    
    return login_layout, "Invalid username or password"
@app.callback(
    Output('breed-dropdown', 'options'),
    Input('filter-type', 'value')
)
def populate_breed_dropdown(filter_type):
    sorted_breeds = sorted(breed_index.keys())
    return [{'label': breed, 'value': breed} for breed in sorted_breeds]

# CSV export callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("btn_csv", "n_clicks"),
    State('datatable-id', "derived_virtual_data"),
    prevent_initial_call=True
)
def download_csv(n_clicks, view_data):
    if not view_data:
        return
    df = pd.DataFrame.from_dict(view_data)
    return dcc.send_data_frame(df.to_csv, "filtered_animals.csv", index=False)

# JSON export callback
@app.callback(
    Output("download-dataframe-json", "data"),
    Input("btn_json", "n_clicks"),
    State('datatable-id', "derived_virtual_data"),
    prevent_initial_call=True
)
def download_json(n_clicks, view_data):
    if not view_data:
        return
    df = pd.DataFrame.from_dict(view_data)
    return dict(content=df.to_json(orient='records', indent=2), filename="filtered_animals.json")

#############################################
# Interaction Between Components / Controller
#############################################
# Then inside the Dash callback:
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'),
     Input('sort-dropdown', 'value'),
     Input('breed-dropdown', 'value'),
     Input('age-range-slider', 'value')]
)
def update_dashboard(filter_type, sort_by, selected_breed, age_range):
    try:
        query = build_query(filter_type)
        df_filtered = filter_and_sort_data(
            db, query,
            sort_by=sort_by,
            breed_index=breed_index,
            filter_breed=selected_breed
        )

        age_filtered = BST()
        for _, row in df_filtered.iterrows():
            if pd.notnull(row["age_upon_outcome_in_weeks"]):
                age_filtered.insert(row["age_upon_outcome_in_weeks"], row.to_dict())
        filtered_records = age_filtered.range_query(age_range[0], age_range[1])

        # Convert back to DataFrame to apply final sorting
        df_final = pd.DataFrame.from_records(filtered_records)

        # Apply final sorting
        if sort_by and sort_by in df_final.columns:
            df_final = df_final[df_final[sort_by].notnull()]
    
        if sort_by == 'age_upon_outcome_in_weeks':
            df_final[sort_by] = pd.to_numeric(df_final[sort_by], errors='coerce')
        else:
            df_final[sort_by] = df_final[sort_by].astype(str)

        df_final.sort_values(by=sort_by, ascending=True, inplace=True)

        return df_final.to_dict('records')

    except Exception as e:
        print("Callback Error:", e)
        return []

# Callback to update the pie chart
@app.callback(Output('graph-id', "children"), [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if not viewData:
        return [html.P("No data available", style={'textAlign': 'center', 'color': 'red'})]
    dff = pd.DataFrame.from_dict(viewData)
    if 'breed' not in dff:
        return [html.P("Breed data missing", style={'textAlign': 'center', 'color': 'red'})]
    fig = px.pie(dff, names='breed', title='Preferred Animal Breeds')
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns: 
        return []
    return [{'if': {'column_id': i}, 'backgroundColor': '#D2F3FF'} for i in selected_columns]

# Callback to update the map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    
    # Updates the map based on the selected dog.
    
    if not viewData or not index:
        return [html.P("No data available", style={'textAlign': 'center', 'color': 'red'})]
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index and index[0] < len(dff) else 0
    lat = dff.iloc[row].get("location_lat", 30.75)
    lon = dff.iloc[row].get("location_long", -97.48)

    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        dl.Marker(position=[lat, lon], children=[
            dl.Tooltip(dff.iloc[row].get("breed", "Unknown")),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row].get("name", "Unknown"))
            ])
        ])
    ])]

app.run_server(mode='external', debug=True)

print(f"Total records: {len(df)}")

C:\dash_env\venv\Lib\site-packages\dash\dash.py:516: UserWarning: JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.
  warnings.warn(


Dash app running on http://127.0.0.1:8050/
Total records: 10000
Callback Error: None
Callback Error: None
Callback Error: None
Callback Error: None
Callback Error: None
Callback Error: None
Callback Error: None
Callback Error: None
Callback Error: None
Callback Error: None
Callback Error: None
